In [1]:
import numpy as np
import pandas as pd

import lingd

from IPython.display import display, Markdown

np.random.seed(0)

## Test data

### Adjacency matrices

In [2]:
B = np.array([
    [
        [ 0.00000, 0.00000, 0.07599, 0.00000, 0.00000,-0.19593, 0.00000, 0.13989],
        [ 0.01176, 0.00000, 0.18145, 0.00000, 0.00000,-0.29303, 0.08484,-0.22178],
        [ 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000,-0.06148, 0.00000],
        [ 0.20774, 0.07962, 0.12428, 0.00000, 0.00000,-0.09539, 0.16998, 0.00000],
        [ 0.20022,-0.21490,-0.27879, 0.08682, 0.00000, 0.00000, 0.03954, 0.00000],
        [ 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000],
        [ 0.02913, 0.00000, 0.00000, 0.00000, 0.00000, 0.05897, 0.00000, 0.24630],
        [ 0.00000, 0.00000, 0.00000, 0.00000, 0.00000,-0.09463, 0.00000, 0.00000],
    ],
    [
        [ 0.00000, 0.00000, 0.11504, 0.08206, 0.00000, 0.00000, 0.00000,-0.17650],
        [ 0.12741,-0.13907, 0.00770, 0.00000, 0.00000, 0.00000,-0.11225,-0.06224],
        [ 0.05388, 0.00000, 0.20890, 0.20382, 0.13611, 0.09354, 0.00000, 0.00000],
        [ 0.00000, 0.00000, 0.00000,-0.16963, 0.00000, 0.00000, 0.21506, 0.00000],
        [ 0.00000,-0.22460, 0.00000, 0.27112,-0.02503, 0.00000, 0.00000,-0.10668],
        [ 0.00000, 0.00000, 0.28331, 0.00000, 0.13032, 0.05586, 0.00000, 0.22187],
        [ 0.00000, 0.00000,-0.28584, 0.07618, 0.00000, 0.00000,-0.28421, 0.00000],
        [-0.06507,-0.07900, 0.26900, 0.00620, 0.22751, 0.00000,-0.19892, 0.00000],
    ],
    np.array([
        [ 0.00000,  0.00000, -0.57550,  0.57112,  0.34546,  0.00000,  0.00000,  0.27488],
        [-0.57538,  0.00000,  0.00000,  0.00000,  0.11114, -0.15522,  0.00000, -0.01998],
        [ 0.00000,  0.00000,  0.00000,  0.00000, -0.23942,  0.11228,  0.52922,  0.32628],
        [ 0.00000,  0.00000,  0.00000,  0.19602, -0.13964,  0.00000,  0.52170,  0.00000],
        [-0.25728, -0.15116, -0.01984,  0.13270,  0.00000, -0.36868,  0.04084,  0.00936],
        [-0.40884, -0.07732,  0.00000,  0.00000,  0.00000,  0.00000, -0.16656,  0.00000],
        [ 0.09564,  0.00000, -0.54530,  0.03762,  0.00000,  0.49246,  0.00000,  0.00000],
        [ 0.00000,  0.00000, -0.41402, -0.14402,  0.00000,  0.00000,  0.45406,  0.41978],
    ]),
])

n_taus, n_features, _ = B.shape

### Data Generation

In [3]:
T = 100
k = n_taus - 1

I_minus_B0_inv = np.linalg.pinv(np.eye(n_features) - B[0])

X = np.empty((T * 2, n_features))

for t in range(T * 2):
    lag = min(k, t)

    lag_data = np.array(X[t - lag:t])
    if len(lag_data) == 0:
        lag_data = 0
    else:
        lag_data = np.hstack(B[1:lag+1]) @ np.hstack([*lag_data[::-1]]).reshape(-1, 1)

    e = np.random.uniform(-1, 1, size=(n_features, 1))
    X[t] = (I_minus_B0_inv @ (lag_data + e)).T

X = X[-T:]

## Causal Discovery

CyclicVARLiNGAM estimates a causal structure from data X. The argument k is the number of lags.

In [4]:
model = lingd.CyclicVARLiNGAM(lags=2)
model.fit(X)

/home/haraoka/.local/lib/python3.11/site-packages/sklearn/decomposition/_fastica.py:127: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(


The estimated adjacency matrices are as follows:

In [5]:
for t, adj in enumerate(model.adjacency_matrices_):
    display(Markdown(f"#### t={t}"))
    display(pd.DataFrame(adj))

#### t=0

,0,1,2,3,4,5,6,7
0,0.000000,0.465582,0.176040,0.524395,-0.400482,0.746700,-0.353679,0.255800
1,-0.303694,0.000000,-0.362638,0.304878,-0.075395,-0.642503,-1.604137,1.501470
2,0.246058,0.890653,0.000000,0.148210,0.250538,0.403259,-0.229440,0.471490
3,0.208712,0.001469,-0.345882,0.000000,-0.341473,-0.005054,-0.437085,-0.345482
4,0.588407,-0.166195,-0.487626,0.183006,0.000000,0.358145,0.038252,0.250823
5,-0.593996,0.461903,0.137443,-0.076458,-0.535848,0.000000,0.185388,0.186210
6,1.239137,0.848685,1.355663,-0.149625,-0.216457,0.246121,0.000000,-0.066570
7,-0.172686,-0.134654,-0.314046,1.350682,-0.099657,-0.178586,-0.108146,0.000000


#### t=1

,0,1,2,3,4,5,6,7
0,-0.274330,0.168686,0.011796,0.119307,0.029591,-0.151199,0.136537,-0.347868
1,0.070172,-0.202819,-0.283826,0.316303,0.064361,-0.515304,-0.038363,0.104667
2,0.030261,0.151650,-0.124375,0.169612,-0.085870,0.066045,-0.109008,0.070605
3,0.087373,-0.124839,-0.007732,-0.086559,0.287786,0.125093,-0.170606,-0.578060
4,-0.199781,-0.127399,-0.218538,0.391399,-0.148033,0.044248,0.115761,-0.154389
5,-0.285676,0.043127,0.607711,-0.097277,0.418922,0.021715,0.496777,0.072999
6,-0.450387,0.257560,-0.196127,-0.062484,-0.345676,-0.604038,-0.257060,0.775929
7,-0.080391,0.064262,0.090228,0.615372,0.265844,0.148884,-0.647074,0.317415


You also have access to the candidates and whether the candidates are stable or not and the costs of the candidates.

In [6]:
for adj, is_stable, cost in zip(model.adjacency_matrices_list_, model.is_stables_, model.costs_):
    print(adj.shape, is_stable, cost)

(2, 8, 8) False 7.45022936091234
(2, 8, 8) False 8.12962268565945
(2, 8, 8) False 8.29684206877316
(2, 8, 8) False 8.4094584366456
(2, 8, 8) False 8.46187465899845


CyclicVARLiNGAM selects a stable candidate or, if no stable candidate is available, selects the candidate with the lowest cost and sets it to model.adacnecy_matrices_.

## Bootstrapping

In [7]:
n_sampling = 100
bs_result = model.bootstrap(X, n_sampling)

/home/haraoka/.local/lib/python3.11/site-packages/sklearn/decomposition/_fastica.py:127: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(
/home/haraoka/.local/lib/python3.11/site-packages/sklearn/decomposition/_fastica.py:127: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(
/home/haraoka/.local/lib/python3.11/site-packages/sklearn/decomposition/_fastica.py:127: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(
/home/haraoka/.local/lib/python3.11/site-packages/sklearn/decomposition/_fastica.py:127: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(
/home/haraoka/.local/lib/python3.11/site-packages/sklearn/decomposition/_fastica.py:127: ConvergenceWarning: FastICA did not